In [1]:
!pip install mysql-connector-python

In [2]:
#TASK1
import mysql.connector as connector
connection=connector.connect(user="username",password="password")

In [3]:
from mysql.connector.pooling import MySQLConnectionPool

In [4]:
from mysql.connector import Error

In [5]:
dbconfig = {
    "database":"little_lemon_db",
    "user" : "username",
    "password" : "password"
}

In [6]:
try:
    pool = MySQLConnectionPool(pool_name = "pool_b",pool_size = 2,**dbconfig)
    print("The connection pool is created with the name: ",pool.pool_name)
    print("The pool size is:",pool.pool_size)

except Error as er:
    print("Error code:", er.errno)
    print("Error message:", er.msg)

The connection pool is created with the name:  pool_b
The pool size is: 2


In [7]:
#Task2
connection1 = pool.get_connection()
cursor1=connection1.cursor()
booking1="""INSERT INTO Bookings 
(TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES
(8,'Anees','Java','18:00:00',6);"""
cursor1.execute(booking1)
connection1.commit()
print("""A new booking is added in the "Bookings" table.""")

connection2 = pool.get_connection()
cursor2=connection2.cursor()
booking2="""INSERT INTO Bookings 
(TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES
(5, 'Bald','Vin','19:00:00',6);"""
cursor2.execute(booking2)
connection2.commit()
print("""A new booking is added in the "Bookings" table.""")

import mysql.connector as connector
try:
    connection3 = pool.get_connection()
    print("The guest is connected")
except:
    print("Adding new connection in the pool.")
        
    connection=connector.connect(user="username",password="password")
    pool.add_connection(cnx=connection)
    print("A new connection is added in the pool.\n")
        
    connection3 = pool.get_connection()
    print("'connection3' is added in the pool.")
    
cursor3=connection3.cursor()
booking3="""INSERT INTO Bookings 
(TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES
(12, 'Jay','Kon','19:30:00',6);"""
cursor3.execute(booking3)
connection3.commit()
print("""A new booking is added in the "Bookings" table.""")

from mysql.connector import Error
for connection in [connection1,connection2, connection3]:
    try:
        connection.close()
        print("Connection is returned to the pool")
    except Error as er:
        print("\nConnection can't be returned to the pool")
        print("Error message:", er.msg)

A new booking is added in the "Bookings" table.
A new booking is added in the "Bookings" table.
Adding new connection in the pool.
A new connection is added in the pool.

'connection3' is added in the pool.
A new booking is added in the "Bookings" table.
Connection is returned to the pool
Connection is returned to the pool

Connection can't be returned to the pool
Error message: Failed adding connection; queue is full


In [8]:
#TASK3
print("Getting a connection from the pool.")
connection = pool.get_connection()
print("""The object "connection" is created with a connection link from the pool_a""")
print("""Creating a cursor object to communicate with the database.""")
cursor=connection.cursor()
print("""The cursor object "cursor" is created.""")

cursor.execute("""
SELECT 
Name, EmployeeID 
FROM Employees 
WHERE Role = 'Manager'
""")
results=cursor.fetchall()
columns=cursor.column_names
print(columns)
for result in results:
    print(result)

cursor.execute("""SELECT 
Name, EmployeeID 
FROM Employees ORDER BY 
Annual_Salary DESC LIMIT 1""")
results=cursor.fetchall()
columns=cursor.column_names
print(columns)
for result in results:
    print(result)

cursor.execute("""SELECT 
COUNT(BookingID) n_booking_between_18_20_hrs
FROM Bookings 
WHERE BookingSlot BETWEEN '18:00:00' AND '20:00:00';""")
results=cursor.fetchall()
columns=cursor.column_names
print(columns)
for result in results:
    print(result)


cursor.execute("""
SELECT 

Bookings.BookingID AS ID,  
CONCAT(GuestFirstName,' ',GuestLastName) AS GuestName, 

Role AS Employee

FROM Bookings 
LEFT JOIN 
Employees 
ON Employees.EmployeeID=Bookings.EmployeeID
WHERE Employees.Role = "Receptionist"
ORDER BY BookingSlot DESC;

""")
print("The following guests are waiting to be seated:")
results=cursor.fetchall()
columns=cursor.column_names
print(columns)
for result in results:
    print(result)


Getting a connection from the pool.
The object "connection" is created with a connection link from the pool_a
Creating a cursor object to communicate with the database.
The cursor object "cursor" is created.
('Name', 'EmployeeID')
('Mario Gollini', 1)
('Name', 'EmployeeID')
('Mario Gollini', 1)
('n_booking_between_18_20_hrs',)
(13,)
The following guests are waiting to be seated:
('ID', 'GuestName', 'Employee')
(7, 'Jay Kon', 'Receptionist')
(10, 'Jay Kon', 'Receptionist')
(13, 'Jay Kon', 'Receptionist')
(4, 'Bald Vin', 'Receptionist')
(6, 'Bald Vin', 'Receptionist')
(9, 'Bald Vin', 'Receptionist')
(12, 'Bald Vin', 'Receptionist')
(1, 'Anees Java', 'Receptionist')
(2, 'Anees Java', 'Receptionist')
(3, 'Anees Java', 'Receptionist')
(5, 'Anees Java', 'Receptionist')
(8, 'Anees Java', 'Receptionist')
(11, 'Anees Java', 'Receptionist')


In [9]:
#TASK4
cursor.execute("DROP PROCEDURE IF EXISTS BasicSalesReport;")

stored_procedure_query="""
CREATE PROCEDURE BasicSalesReport()

BEGIN
SELECT 
SUM(BillAmount) AS Total_Sale,
AVG(BillAmount) AS Average_Sale,
MIN(BillAmount) AS Min_Bill_Paid,
MAX(BillAmount) AS Max_Bill_Paid
FROM Orders;
END
"""

cursor.execute(stored_procedure_query)
cursor.callproc("BasicSalesReport")


results = next(cursor.stored_results())
results = results.fetchall()

for column_id in cursor.stored_results():
    cols = [column[0] for column in column_id.description]

    
print("Today's sales report:")
for result in results:
    print("\t", cols[0], ":", result[0])
    print("\t", cols[1], ":", result[1])
    print("\t", cols[2], ":", result[2])
    print("\t", cols[3], ":", result[3])


Today's sales report:
	 Total_Sale : 243
	 Average_Sale : 48.6000
	 Min_Bill_Paid : 37
	 Max_Bill_Paid : 86


In [10]:
#TASK5
connection = pool.get_connection()
cursor=connection.cursor(buffered=True)

sql_query="""SELECT 
Bookings.BookingSlot,
CONCAT(Bookings.GuestFirstName," ",Bookings.GuestLastName) AS Guest_Name,
Employees.Name AS Emp_Name,
Employees.Role AS Emp_Role
FROM Bookings 
INNER JOIN 
Employees ON Bookings.EmployeeID=Employees.EmployeeID
ORDER BY Bookings.BookingSlot ASC;"""
cursor.execute(sql_query)
results=cursor.fetchmany(size=3)
#print(cursor.column_names)
for result in results:
    print("\nBookingSlot",result[0])
    print("\tGuest_name:",result[1])
    print("\tAssigned to:",result[2],"[{}]".format(result[3]))
    
connection.close()


BookingSlot 18:00:00
	Guest_name: Anees Java
	Assigned to: John Millar [Receptionist]

BookingSlot 18:00:00
	Guest_name: Anees Java
	Assigned to: John Millar [Receptionist]

BookingSlot 18:00:00
	Guest_name: Anees Java
	Assigned to: John Millar [Receptionist]
